In [1]:
from lightning_modules import (
    AnyOrderInsertionFlowModule,
)
from sampling import any_order_mask_insertion_euler_sampling, SamplingTraceDatapoint

In [2]:
!nvidia-smi

Thu Jun 26 18:12:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  |   00000000:E3:00.0 Off |                    0 |
| N/A   27C    P0             51W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
checkpoint_path = "/n/netscratch/albergo_lab/Lab/brianlck/interpretable-flow/checkpoints/bracket/any_order/20250626-172722/epoch-epoch=06-val_loss-val_loss=0.6431.ckpt"

model = AnyOrderInsertionFlowModule.load_from_checkpoint(checkpoint_path)
model.swap_to_ema()

In [4]:
steps = 5
batch_size = 20

samples, trace = any_order_mask_insertion_euler_sampling(
    model,
    model.interpolant,
    steps=steps,
    mask=0,
    pad=3,
    batch_size=batch_size,
    max_length=64,
    return_trace=True,
)

tensor([[14.3796,  0.1804,  0.2464,  ...,  0.2542,  0.2464,  0.2623],
        [17.6581,  0.2175,  0.2315,  ...,  0.2175,  0.1804,  0.1920],
        [11.4140,  0.2464,  0.2315,  ...,  0.2464,  0.2542,  0.2315],
        ...,
        [13.0042,  0.2175,  0.2792,  ...,  0.2464,  0.1543,  0.2315],
        [12.7257,  0.1592,  0.1861,  ...,  0.2175,  0.1920,  0.1362],
        [10.1542,  0.2623,  0.2043,  ...,  0.2244,  0.1981,  0.2244]],
       device='cuda:0', grad_fn=<MulBackward0>)
tensor([], device='cuda:0', dtype=torch.int64) tensor([], device='cuda:0', dtype=torch.int64)
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19], device='cuda:0') tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')
tensor([[4.7290, 5.6787, 0.1459,  ..., 0.2057, 0.1994, 0.1994],
        [5.1827, 6.7108, 0.1994,  ..., 0.1994, 0.1994, 0.2057],
        [5.0269, 5.5929, 0.1653,  ..., 0.2190, 0.2057, 0.1816],
        ...,
        [4.3

In [14]:
def process_trace(trace: list[SamplingTraceDatapoint]):
    event_type_mapping = dict(change="c", insertion="i")
    token_mapping = {0: "m", 1: "(", 2: ")"}

    def _process_datapoint(datapoint: SamplingTraceDatapoint):
        return dict(
            t=datapoint.t,
            a=event_type_mapping[datapoint.event_type],
            tk=token_mapping[datapoint.token],
            i=datapoint.position,
        )

    return [_process_datapoint(datapoint) for datapoint in trace]

In [34]:
from IPython.display import display, HTML
import json

# Select the index of the tensor to visualize
id = 19
assert 0 <= id < batch_size, "id must be in [0, batch_size)"

events = process_trace(trace[id])
print(events)

display(
    HTML(f"""
<style>
.token {{
    display: inline-block;
    padding: 5px 10px;
    margin: 2px;
    border: 1px solid #aaa;
    border-radius: 4px;
    font-family: monospace;
    transition: background 0.5s;
}}
.hi-i {{ background: lightgreen; }}
.hi-c {{ background: yellow; }}
#s {{ min-height: 30px; margin-bottom: 20px; }}
</style>
<div id="s"></div>
<button id="r">Replay</button>
<script>
var ev = {json.dumps(events)};
var s = [];
var delay = 100; // Delay between steps (ms)

function render(hIdx, action) {{
    var container = document.getElementById("s");
    container.innerHTML = "";
    s.forEach(function(token, idx) {{
        var span = document.createElement("span");
        span.className = "token" + (idx === hIdx ? " hi-" + action : "");
        span.textContent = token;
        container.appendChild(span);
    }});
}}

async function run() {{
    s = [];
    render();
    
    for (const e of ev) {{
        if (e.a === "i") {{
            // Insert token at position e.i
            var pos = (e.i < 0 || e.i > s.length) ? Math.floor(s.length / 2) : e.i;
            s.splice(pos, 0, e.tk);
            render(pos, "i");
        }} else {{
            // Change token at position e.i
            var pos = (e.i === undefined || e.i < 0 || e.i >= s.length) ? Math.floor(s.length / 2) : e.i;
            s[pos] = e.tk;
            render(pos, "c");
        }}
        
        // Small delay to make the animation visible (but still synchronous in execution order)
        await new Promise(resolve => setTimeout(resolve, delay));
    }}
}}

document.getElementById("r").onclick = run;
run(); // Start automatically
</script>
""")
)

KeyError: 3